<a href="https://colab.research.google.com/github/khauzenberger/pytorch-projects/blob/main/1_long_term_forecasting_with_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The project

In a frist step, I'm going to be replicating the paper "A Time Series is Worth 64 Words: Long-term Forecasting with Transformers" by Yuqi Nie, Nam H. Nguyen, Phanwadee Sinthong and Jayant Kalagnanam (see https://arxiv.org/abs/2211.14730).

In the second step, then, I apply the key designs of the paper to forecast macroeconomic time series.

## Key designs, model overview, and key concept

**Patching**: Segmentation of time series into subseries-level patches which are served as input tokens to Transformer.

**Channel-independence**: Each channel contains a single univariate time series that shares the same embedding and Transformer weights across all the series.

The figure below sketches the so-called **PatchTST model** and transformer backbones under supervised and self-supervised learning.

![picture](https://raw.githubusercontent.com/yuqinie98/PatchTST/main/pic/model.png)

**Transformer-based models**: It is a deep learning model that adopts the mechanism of self-attention, differentially weighting the significance of each part of the input data.

Transformers are designed to process sequential input data, processing them all at once. The attention mechanism provides context for any position in the input sequence. 

Because transformers process the entire input all at once, they allow for more parallelization than recurrent neural networks (RNNs) and therefore reduce training times.

# Replication

In the following I provide step-by-step instructions to get from our inputs to the desired outputs.

While the paper comes with an official implementation (https://github.com/yuqinie98/PatchTST), there's no learning in just copying and pasting them. That's why I will start more or less from scratch, going through the steps in the section on paper replication in Daniel Bourke's ZTM course "PyTorch for deep learning" (https://github.com/khauzenberger/pytorch-deep-learning). 

## Get data from the paper

While the paper experiments with eight different datasets, I will focus only on the smallest one: the influenza-like illnesses (ILI) dataset. 

In [1]:
import requests
from pathlib import Path

# Setup path to data folder
data_path = Path("data/")

# If the image folder doesn't exist, download it and prepare it... 
if data_path.is_dir():
    print(f"{data_path} directory exists.")
else:
    print(f"Did not find {data_path} directory, creating one...")
    data_path.mkdir(parents=True, exist_ok=True)
    
    # Download influenza-like illness data from my Github repo
    with open(data_path / "national_illness.csv", "wb") as f:
        request = requests.get("https://github.com/khauzenberger/pytorch-projects/raw/main/data/national_illness.csv")      
        print("Downloading influenza-like illness data from my Github repo...")
        f.write(request.content)

Did not find data directory, creating one...


## Create Datasets and DataLoaders

In [2]:
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader

def create_dataloaders(data_path:str, file_name:str, seq_len:int, pred_len:int, 
                       batch_size:int=1, scale=False,features:str="S",
                       target:str="OT"):
  
  scaler = StandardScaler()
  df_raw = pd.read_csv(os.path.join(data_path,file_name))
  
  cols = list(df_raw.columns)
  cols.remove(target)
  cols.remove("date")
  df_raw = df_raw[["date"] + cols + [target]]

  num_train = int(len(df_raw) * 0.7)
  num_test = int(len(df_raw) * 0.2)
  num_vali = len(df_raw) - num_train - num_test

  border1s = [0, num_train - seq_len, len(df_raw) - num_vali - seq_len]
  border2s = [num_train, num_train + num_test, len(df_raw)]

  if features == "M" or features == "MS":
    cols_data = df_raw.columns[1:]
    df_data = df_raw[cols_data]
  elif features == "S":
    df_data = df_raw[[target]]

  if scale:
    train_data = df_data[border1s[0]:border2s[0]]
    scaler.fit(train_data.values)
    data = scaler.transform(df_data.values)
  else:
    data = df_data.values

  train_data = data[border1s[0]:border2s[0]]
  test_data = data[border1s[1]:border2s[1]]
  vali_data = data[border1s[2]:border2s[2]]

  train_dataloader = DataLoader(
      train_data,
      batch_size=batch_size,
      shuffle=True)
  test_dataloader = DataLoader(
      test_data,
      batch_size=batch_size,
      shuffle=False)
  vali_dataloader = DataLoader(
      vali_data,
      batch_size=batch_size,
      shuffle=False)

  return train_dataloader, test_dataloader, vali_dataloader

## Model settings

In [3]:
# Setup path to data folder and file name
data_path = Path("data/")
file_name = "national_illness.csv"

# Sequence length (look-back window)
seq_len = 104

# Prediciton length (forecast horizon)
pred_len = 24

# Batch size
batch_size = 16

# Features of model: M  -> multivariate predict multivariate
#                    S  -> univariate predict univariate
#                    MS -> multivariate predict univariate
features = "M"

# Various parameters and hyperparameters
d_model = 16
patch_len = 24
stride = 2
n_heads = 4
enc_in = 7
e_layers = 3
d_ff = 128
dropout = 0.3
fc_dropout = 0.3
head_dropout = 0
learning_rate = 0.0025
padding_patch="end"

## PatchTST architecture

### Instance normalization

This technique helps mitigating the distribution shift effect between the training and testing data.

In [4]:
import torch
import torch.nn as nn

class InstanceNorm(nn.Module):
  def __init__(self, num_features:int, eps=1e-5,  affine=True, 
               subtract_last=False):    
    """
    :param num_features: the number of features or channels
    :param eps: a value added for numerical stability
    :param affine: if True, InstanceNorm has learnable affine parameters
    """    
    super(InstanceNorm, self).__init__()
    self.num_features = num_features
    self.eps = eps
    self.affine = affine
    self.subtract_last = subtract_last
    if self.affine:
      self._init_params()

  def forward(self, x, mode:str):
    if mode == "norm":
      self._get_statistics(x)
      x = self._normalize(x)
    elif mode == "denorm":
      x = self._denormalize(x)
    else: raise NotImplementedError
    return x

  def _init_params(self):
    # initialize InstanceNorm params: (C,)
    self.affine_weight = nn.Parameter(torch.ones(self.num_features))
    self.affine_bias = nn.Parameter(torch.zeros(self.num_features))

  def _get_statistics(self, x):
    dim2reduce = tuple(range(1, x.ndim-1))
    if self.subtract_last:
      self.last = x[:,-1,:].unsqueeze(1)
    else:
      self.mean = torch.mean(x, dim=dim2reduce, keepdim=True).detach()
      self.stdev = torch.sqrt(torch.var(x, dim=dim2reduce, keepdim=True, unbiased=False) + self.eps).detach()

  def _normalize(self, x):
    if self.subtract_last:
      x = x - self.last
    else:
      x = x - self.mean
      x = x / self.stdev
    if self.affine:
      x = x * self.affine_weight
      x = x + self.affine_bias
    return x

  def _denormalize(self, x):
    if self.affine:
      x = x - self.affine_bias
      x = x / (self.affine_weight + self.eps*self.eps)
      x = x * self.stdev
    if self.subtract_last:
      x = x + self.last
    else:
      x = x + self.mean
    return x

In [ ]:
random_data = torch.randn(10,7)
instnorm = InstanceNorm(num_features=7)
instnorm(random_data,"norm")

## Patching and Padding

Pads the input tensor using replication of the input boundary.



In [ ]:
from numpy.lib.arraypad import pad
import torch
import torch.nn as nn

patch_num = int((seq_len-patch_len)/stride+1)
if padding_patch == "end":
  padding_patch_layer = nn.ReplicationPad1d((0, stride))
  patch_num += 1

### Input encoding (projection)

Implemeting equation 1: Projection of feature vectors onto a d-dim vector space.

In [ ]:
W_P = nn.Linear(patch_len, d_model)

### Positional encoding (position embedding)

In [5]:
import torch
import torch.nn as nn
import math
from typing import Callable, Optional
from torch import Tensor
import torch.nn.functional as F
import numpy as np

def PositionalEncoding(pe, learn_pe, q_len, d_model):
    if pe == None:
      W_pos = torch.empty((q_len, d_model))
      nn.init.uniform_(W_pos, -0.02, 0.02)
      learn_pe = False
    elif pe == "zero":
        W_pos = torch.empty((q_len, 1))
        nn.init.uniform_(W_pos, -0.02, 0.02)
    elif pe == "zeros":
        W_pos = torch.empty((q_len, d_model))
        nn.init.uniform_(W_pos, -0.02, 0.02)
    elif pe == "normal" or pe == "gauss":
        W_pos = torch.zeros((q_len, 1))
        torch.nn.init.normal_(W_pos, mean=0.0, std=0.1)
    elif pe == "uniform":
        W_pos = torch.zeros((q_len, 1))
        nn.init.uniform_(W_pos, a=0.0, b=0.1)
    else: raise ValueError(f"{pe} is not a valid pe (positional encoder). Available types: 'gauss'=='normal', \
        'zeros', 'zero', 'uniform'.")
    return nn.Parameter(W_pos, requires_grad=learn_pe)

### Backbone

In [12]:
from typing import Callable, Optional
import torch
from torch import nn
from torch import Tensor
import torch.nn.functional as F
import numpy as np

class TransformerBackbone(nn.Module):
  def __init__(self, c_in:int, context_window:int, target_window:int, patch_len:int,
               stride:int, max_seq_len:Optional[int]=1024, n_layers:int=3, d_model=128,
               n_heads=16, d_k:Optional[int]=None, d_v:Optional[int]=None, d_ff:int=256, 
               norm:str='BatchNorm', attn_dropout:float=0., dropout:float=0., act:str="gelu", 
               key_padding_mask:bool='auto', padding_var:Optional[int]=None, 
               attn_mask:Optional[Tensor]=None, res_attention:bool=True, pre_norm:bool=False, 
               store_attn:bool=False, pe:str='zeros', learn_pe:bool=True, fc_dropout:float=0., 
               head_dropout = 0, padding_patch = None, pretrain_head:bool=False, 
               head_type = 'flatten', individual = False, insnorm = True, affine = True, 
               subtract_last = False, verbose:bool=False, **kwargs):
        
    super().__init__()

    # Instance normalization
    self.insnorm = insnorm
    if insnorm:
      self.insnorm_layer = InstanceNorm(c_in, eps=1e-5, affine=affine,
                                        subtract_last=subtract_last)
    
    # Patching and padding
    self.patch_len = patch_len
    self.stride = stride
    self.padding_patch = padding_patch
    patch_num = int((seq_len-patch_len)/stride+1)
    if padding_patch == "end":
      self.padding_patch_layer = nn.ReplicationPad1d((0, stride))
      patch_num += 1

    # Backbone 
    self.backbone = TransformerIndependentEncoder(c_in, patch_num=patch_num, patch_len=patch_len, 
                                                  max_seq_len=max_seq_len, n_layers=n_layers, d_model=d_model, 
                                                  n_heads=n_heads, d_k=d_k, d_v=d_v, d_ff=d_ff,
                                                  attn_dropout=attn_dropout, dropout=dropout, act=act, 
                                                  key_padding_mask=key_padding_mask, padding_var=padding_var,
                                                  attn_mask=attn_mask, res_attention=res_attention, pre_norm=pre_norm, 
                                                  store_attn=store_attn, pe=pe, learn_pe=learn_pe, verbose=verbose, 
                                                  **kwargs)
    
    # Head
    self.head_nf = d_model * patch_num
    self.n_vars = c_in
    self.pretrain_head = pretrain_head
    self.head_type = head_type
    self.individual = individual    

    if self.pretrain_head: 
      # custom head passed as a partial func with all its kwargs
      self.head = self.create_pretrain_head(self.head_nf, c_in, fc_dropout) 
    elif head_type == 'flatten': 
      self.head = FlattenHead(self.individual, self.n_vars, self.head_nf, 
                              target_window, head_dropout=head_dropout)

    def create_pretrain_head(self, head_nf, vars, dropout):
      return nn.Sequential(nn.Dropout(dropout), nn.Conv1d(head_nf, vars, 1))

    # Forward pass
    def forward(self, z):                            
      # Instance normalization
      if self.insnorm:
        z = z.permute(0,2,1)                                             # z: [bs x nvars x seq_len]
        z = self.revin_layer(z, "norm")
        z = z.permute(0,2,1)
            
      # Patching
      if self.padding_patch == "end":
        z = self.padding_patch_layer(z)
      z = z.unfold(dimension=-1, size=self.patch_len, step=self.stride)  # z: [bs x nvars x patch_num x patch_len]           
      z = z.permute(0,1,3,2)                                             # z: [bs x nvars x patch_len x patch_num]                                                             
        
      # Model
      z = self.backbone(z)                                               # z: [bs x nvars x d_model x patch_num]
      z = self.head(z)                                                   # z: [bs x nvars x target_window] 
        
      # Instance denormalization
      if self.insnorm: 
        z = z.permute(0,2,1)
        z = self.revin_layer(z, "denorm")
        z = z.permute(0,2,1)
      return z      

In [10]:
import torch
from torch import nn
from torch import Tensor

class TransformerIndependentEncoder(nn.Module):
  def __init__(self, c_in, patch_num, patch_len, max_seq_len=1024, n_layers=3, 
               d_model=128, n_heads=16, d_k=None, d_v=None,d_ff=256, norm='BatchNorm', 
               attn_dropout=0., dropout=0., act="gelu", store_attn=False,
               key_padding_mask='auto', padding_var=None, attn_mask=None, 
               res_attention=True, pre_norm=False, pe='zeros', learn_pe=True, 
               verbose=False, **kwargs):
               
    super().__init__()    
    
    # Input encoding
    self.W_P = nn.Linear(patch_len, d_model)

    # Positional encoding
    self.W_pos = PositionalEncoding(pe, learn_pe, patch_num, d_model)

    # Residual dropout
    self.dropout = nn.Dropout(dropout)

    # Encoder
    self.encoder = TransformerEncoder(patch_num, d_model, n_heads, d_k=d_k, d_v=d_v, d_ff=d_ff,
                                      norm=norm, attn_dropout=attn_dropout, dropout=dropout,
                                      pre_norm=pre_norm, activation=act, res_attention=res_attention, 
                                      n_layers=n_layers, store_attn=store_attn)
   
    def forward(self, x) -> Tensor:                                            # x: [bs x nvars x patch_len x patch_num]
        
      # Input encoding
      x = x.permute(0,1,3,2)                                                   # x: [bs x nvars x patch_num x patch_len]
      x = self.W_P(x)                                                          # x: [bs x nvars x patch_num x d_model]

      u = torch.reshape(x, (x.shape[0]*x.shape[1],x.shape[2],x.shape[3]))      # u: [bs * nvars x patch_num x d_model]
      u = self.dropout(u + self.W_pos)                                         # u: [bs * nvars x patch_num x d_model]

      # Encoder
      z = self.encoder(u)                                                      # z: [bs * nvars x patch_num x d_model]
      z = torch.reshape(z, (-1,x.shape[1],z.shape[-2],z.shape[-1]))            # z: [bs x nvars x patch_num x d_model]
      z = z.permute(0,1,3,2)                                                   # z: [bs x nvars x d_model x patch_num]
        
      return z   

In [6]:
import torch
import torch.nn as nn

class FlattenHead(nn.Module):
  def __init__(self, individual, n_vars, nf, target_window, head_dropout=0):
    super().__init__()
        
    self.individual = individual
    self.n_vars = n_vars
        
    if self.individual:
      self.linears = nn.ModuleList()
      self.dropouts = nn.ModuleList()
      self.flattens = nn.ModuleList()
      for i in range(self.n_vars):
        self.flattens.append(nn.Flatten(start_dim=-2))
        self.linears.append(nn.Linear(nf, target_window))
        self.dropouts.append(nn.Dropout(head_dropout))
      else:
        self.flatten = nn.Flatten(start_dim=-2)
        self.linear = nn.Linear(nf, target_window)
        self.dropout = nn.Dropout(head_dropout)
            
  def forward(self, x):                     # x: [bs x nvars x d_model x patch_num]
    if self.individual:
      x_out = []
      for i in range(self.n_vars):
        z = self.flattens[i](x[:,i,:,:])    # z: [bs x d_model * patch_num]
        z = self.linears[i](z)              # z: [bs x target_window]
        z = self.dropouts[i](z)
        x_out.append(z)
        x = torch.stack(x_out, dim=1)       # x: [bs x nvars x target_window]
      else:
        x = self.flatten(x)
        x = self.linear(x)
        x = self.dropout(x)
      return x

In [8]:
from typing import Callable, Optional
import torch
from torch import nn
from torch import Tensor
import torch.nn.functional as F
import numpy as np

class TransformerEncoder(nn.Module):
  def __init__(self, q_len, d_model, n_heads, d_k=None, d_v=None, d_ff=None, norm='BatchNorm', 
               attn_dropout=0., dropout=0., activation='gelu',
               res_attention=False, n_layers=1, pre_norm=False, store_attn=False):
    super().__init__()

    self.layers = nn.ModuleList([TransformerEncoderLayer(q_len, d_model, n_heads=n_heads, d_k=d_k, 
                                                         d_v=d_v, d_ff=d_ff, norm=norm, attn_dropout=attn_dropout, 
                                                         dropout=dropout, activation=activation, 
                                                         res_attention=res_attention, pre_norm=pre_norm, 
                                                         store_attn=store_attn) for i in range(n_layers)])
    self.res_attention = res_attention

  def forward(self, src:Tensor, key_padding_mask:Optional[Tensor]=None, attn_mask:Optional[Tensor]=None):
    output = src
    scores = None
    if self.res_attention:
      for mod in self.layers: output, scores = mod(output, prev=scores, key_padding_mask=key_padding_mask, 
                                                   attn_mask=attn_mask)
      return output
    else:
      for mod in self.layers: output = mod(output, key_padding_mask=key_padding_mask, attn_mask=attn_mask)
      return output


class TransformerEncoderLayer(nn.Module):
  def __init__(self, q_len, d_model, n_heads, d_k=None, d_v=None, d_ff=256, store_attn=False,
               norm='BatchNorm', attn_dropout=0, dropout=0., bias=True, activation="gelu", 
               res_attention=False, pre_norm=False):
    super().__init__()
        
    assert not d_model%n_heads, f"d_model ({d_model}) must be divisible by n_heads ({n_heads})"
    d_k = d_model // n_heads if d_k is None else d_k
    d_v = d_model // n_heads if d_v is None else d_v

    # Multi-Head attention
    self.res_attention = res_attention
    self.self_attn = _MultiheadAttention(d_model, n_heads, d_k, d_v, attn_dropout=attn_dropout, 
                                         proj_dropout=dropout, res_attention=res_attention)

    # Add & Norm
    self.dropout_attn = nn.Dropout(dropout)
    if "batch" in norm.lower():
      self.norm_attn = nn.Sequential(Transpose(1,2), nn.BatchNorm1d(d_model), Transpose(1,2))
    else:
      self.norm_attn = nn.LayerNorm(d_model)

    # Position-wise Feed-Forward
    self.ff = nn.Sequential(nn.Linear(d_model, d_ff, bias=bias), get_activation_fn(activation),
                            nn.Dropout(dropout), nn.Linear(d_ff, d_model, bias=bias))

    # Add & Norm
    self.dropout_ffn = nn.Dropout(dropout)
    if "batch" in norm.lower():
      self.norm_ffn = nn.Sequential(Transpose(1,2), nn.BatchNorm1d(d_model), Transpose(1,2))
    else:
      self.norm_ffn = nn.LayerNorm(d_model)

    self.pre_norm = pre_norm
    self.store_attn = store_attn


  def forward(self, src:Tensor, prev:Optional[Tensor]=None, key_padding_mask:Optional[Tensor]=None, 
              attn_mask:Optional[Tensor]=None) -> Tensor:

    # Multi-Head attention sublayer
    if self.pre_norm:
      src = self.norm_attn(src)
    ## Multi-Head attention
    if self.res_attention:
      src2, attn, scores = self.self_attn(src, src, src, prev, key_padding_mask=key_padding_mask, attn_mask=attn_mask)
    else:
      src2, attn = self.self_attn(src, src, src, key_padding_mask=key_padding_mask, attn_mask=attn_mask)
    if self.store_attn:
      self.attn = attn
    ## Add & Norm
    src = src + self.dropout_attn(src2) # Add: residual connection with residual dropout
    if not self.pre_norm:
      src = self.norm_attn(src)

    # Feed-forward sublayer
    if self.pre_norm:
      src = self.norm_ffn(src)
    ## Position-wise Feed-Forward
    src2 = self.ff(src)
    ## Add & Norm
    src = src + self.dropout_ffn(src2) # Add: residual connection with residual dropout
    if not self.pre_norm:
      src = self.norm_ffn(src)

    if self.res_attention:
      return src, scores
    else:
      return src


class _MultiheadAttention(nn.Module):
  def __init__(self, d_model, n_heads, d_k=None, d_v=None, res_attention=False, attn_dropout=0., 
               proj_dropout=0., qkv_bias=True, lsa=False):
    """Multi Head Attention Layer
    Input shape:
    Q:       [batch_size (bs) x max_q_len x d_model]
    K, V:    [batch_size (bs) x q_len x d_model]
    mask:    [q_len x q_len]
    """
    super().__init__()
    d_k = d_model // n_heads if d_k is None else d_k
    d_v = d_model // n_heads if d_v is None else d_v

    self.n_heads, self.d_k, self.d_v = n_heads, d_k, d_v

    self.W_Q = nn.Linear(d_model, d_k * n_heads, bias=qkv_bias)
    self.W_K = nn.Linear(d_model, d_k * n_heads, bias=qkv_bias)
    self.W_V = nn.Linear(d_model, d_v * n_heads, bias=qkv_bias)

    # Scaled Dot-Product Attention (multiple heads)
    self.res_attention = res_attention
    self.sdp_attn = _ScaledDotProductAttention(d_model, n_heads, attn_dropout=attn_dropout, 
                                               res_attention=self.res_attention, lsa=lsa)

    # Poject output
    self.to_out = nn.Sequential(nn.Linear(n_heads * d_v, d_model), nn.Dropout(proj_dropout))


  def forward(self, Q:Tensor, K:Optional[Tensor]=None, V:Optional[Tensor]=None, 
              prev:Optional[Tensor]=None, key_padding_mask:Optional[Tensor]=None, 
              attn_mask:Optional[Tensor]=None):

    bs = Q.size(0)
    if K is None: K = Q
    if V is None: V = Q

    # Linear (+ split in multiple heads)
    q_s = self.W_Q(Q).view(bs, -1, self.n_heads, self.d_k).transpose(1,2)       # q_s    : [bs x n_heads x max_q_len x d_k]
    k_s = self.W_K(K).view(bs, -1, self.n_heads, self.d_k).permute(0,2,3,1)     # k_s    : [bs x n_heads x d_k x q_len] - transpose(1,2) + transpose(2,3)
    v_s = self.W_V(V).view(bs, -1, self.n_heads, self.d_v).transpose(1,2)       # v_s    : [bs x n_heads x q_len x d_v]

    # Apply Scaled Dot-Product Attention (multiple heads)
    if self.res_attention:
      output, attn_weights, attn_scores = self.sdp_attn(q_s, k_s, v_s, prev=prev, key_padding_mask=key_padding_mask, attn_mask=attn_mask)
    else:
      output, attn_weights = self.sdp_attn(q_s, k_s, v_s, key_padding_mask=key_padding_mask, attn_mask=attn_mask)
    # output: [bs x n_heads x q_len x d_v], attn: [bs x n_heads x q_len x q_len], scores: [bs x n_heads x max_q_len x q_len]

    # Back to the original inputs dimensions
    output = output.transpose(1, 2).contiguous().view(bs, -1, self.n_heads * self.d_v) 
    # output: [bs x q_len x n_heads * d_v]
    output = self.to_out(output)

    if self.res_attention: return output, attn_weights, attn_scores
    else: return output, attn_weights


class _ScaledDotProductAttention(nn.Module):
  r"""Scaled Dot-Product Attention module (Attention is all you need by Vaswani et al., 2017) with optional residual attention from previous layer
  (Realformer: Transformer likes residual attention by He et al, 2020) and locality self sttention (Vision Transformer for Small-Size Datasets
  by Lee et al, 2021)"""

  def __init__(self, d_model, n_heads, attn_dropout=0., res_attention=False, lsa=False):
    super().__init__()
    self.attn_dropout = nn.Dropout(attn_dropout)
    self.res_attention = res_attention
    head_dim = d_model // n_heads
    self.scale = nn.Parameter(torch.tensor(head_dim ** -0.5), requires_grad=lsa)
    self.lsa = lsa

  def forward(self, q:Tensor, k:Tensor, v:Tensor, prev:Optional[Tensor]=None, 
              key_padding_mask:Optional[Tensor]=None, attn_mask:Optional[Tensor]=None):
    '''
    Input shape:
      q               : [bs x n_heads x max_q_len x d_k]
      k               : [bs x n_heads x d_k x seq_len]
      v               : [bs x n_heads x seq_len x d_v]
      prev            : [bs x n_heads x q_len x seq_len]
      key_padding_mask: [bs x seq_len]
      attn_mask       : [1 x seq_len x seq_len]
    Output shape:
      output:  [bs x n_heads x q_len x d_v]
      attn   : [bs x n_heads x q_len x seq_len]
      scores : [bs x n_heads x q_len x seq_len]
    '''

    # Scaled MatMul (q, k) - similarity scores for all pairs of positions in an input sequence
    attn_scores = torch.matmul(q, k) * self.scale      # attn_scores : [bs x n_heads x max_q_len x q_len]

    # Add pre-softmax attention scores from the previous layer (optional)
    if prev is not None: attn_scores = attn_scores + prev

    # Attention mask (optional)
    if attn_mask is not None:                                     # attn_mask with shape [q_len x seq_len] - only used when q_len == seq_len
      if attn_mask.dtype == torch.bool:
        attn_scores.masked_fill_(attn_mask, -np.inf)
      else:
        attn_scores += attn_mask

    # Key padding mask (optional)
    if key_padding_mask is not None:                              # mask with shape [bs x q_len] (only when max_w_len == q_len)
      attn_scores.masked_fill_(key_padding_mask.unsqueeze(1).unsqueeze(2), -np.inf)

    # Normalize the attention weights
    attn_weights = F.softmax(attn_scores, dim=-1)                 # attn_weights   : [bs x n_heads x max_q_len x q_len]
    attn_weights = self.attn_dropout(attn_weights)

    # Compute the new values given the attention weights
    output = torch.matmul(attn_weights, v)                        # output: [bs x n_heads x max_q_len x d_v]

    if self.res_attention: return output, attn_weights, attn_scores
    else: return output, attn_weights


class Transpose(nn.Module):
  def __init__(self, *dims, contiguous=False): 
    super().__init__()
    self.dims, self.contiguous = dims, contiguous
  def forward(self, x):
    if self.contiguous: return x.transpose(*self.dims).contiguous()
    else: return x.transpose(*self.dims)

    
def get_activation_fn(activation):
  if callable(activation): return activation()
  elif activation.lower() == "relu": return nn.ReLU()
  elif activation.lower() == "gelu": return nn.GELU()
  raise ValueError(f'{activation} is not available. You can use "relu", "gelu", or a callable') 
           